In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as O
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.signal import butter, lfilter, freqz
from copy import deepcopy

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device.upper()}")

%matplotlib inline

In [ ]:
rawData = pd.read_csv("dataset.csv")
rawData=rawData.drop(columns="Time in s")
whMapping = {"Person A":[50, 145], "Person G":[58, 168], "Person F":[78.6, 159], "Person H":[58, 168], "Person E": [75, 158], "Person B": [86, 166], "Person C": [65, 174], "Person I":[56, 160], "Person J":[65, 161], "Person D": [70, 161]}
print(whMapping)

In [ ]:
rawData

In [ ]:
# Action coding:
# 1 is flexion
# 0 is extension

def butter_lowpass(cutoff, fs, order=5):
    return butter(order, cutoff, fs=fs, btype='low', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

order=1
fs=1
cutoff=0.1
sections = [[5,44, 1], [50,89, 0], [100,139, 1], [145,184, 0], [195,234, 1], [240,279, 0]]


trainFlexData = {"weight":[], "height":[], "current":[]
             ,"action":[]
            }

testFlexData = {"weight":[], "height":[], "current":[]
             ,"action":[]
            }

trainExtnData = {"weight":[], "height":[], "current":[]
             ,"action":[]
            }

testExtnData = {"weight":[], "height":[], "current":[]
             ,"action":[]
            }

def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.get_cmap(name, n)


cmap = get_cmap(9)
for idx, i in enumerate(list(["Person A","Person B","Person G","Person D","Person E","Person F","Person H","Person I","Person J"])):
    for sec in sections:
        if sec[2]==1:
            continue
        sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
        plt.plot(sig, color=cmap(idx))
        trainFlexData["current"].append(sig)
        trainFlexData["height"].append(whMapping[i][1])
        trainFlexData["weight"].append(whMapping[i][0])
        trainFlexData["action"].append(sec[2])


for i in list(["Person C"]):
    for sec in sections:
        if sec[2] == 1:
            continue
        sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
        testFlexData["current"].append(sig)
        testFlexData["height"].append(whMapping[i][1])
        testFlexData["weight"].append(whMapping[i][0])
        testFlexData["action"].append(sec[2])

for idx, i in enumerate(list(["Person A","Person B","Person G","Person D","Person E","Person F","Person H","Person I","Person J"])):
    for sec in sections:
        if sec[2]==0:
            continue
        sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
        plt.plot(sig, color=cmap(idx))
        trainExtnData["current"].append(sig)
        trainExtnData["height"].append(whMapping[i][1])
        trainExtnData["weight"].append(whMapping[i][0])
        trainExtnData["action"].append(sec[2])


for i in list(["Person C"]):
    for sec in sections:
        if sec[2] == 0:
            continue
        sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
        testExtnData["current"].append(sig)
        testExtnData["height"].append(whMapping[i][1])
        testExtnData["weight"].append(whMapping[i][0])
        testExtnData["action"].append(sec[2])



trainFlexData["current"] = np.array(trainFlexData["current"]) / 1000
testFlexData["current"] = np.array(testFlexData["current"]) / 1000

trainExtnData["current"] = np.array(trainExtnData["current"]) / 1000
testExtnData["current"] = np.array(testExtnData["current"]) / 1000

print("Flexion Statistics")

for i in trainFlexData.keys():
    print(i, len(trainFlexData[i]))
print(np.max(trainFlexData["current"]))
print(np.min(trainFlexData["current"]))
print()

for i in testFlexData.keys():
    print(i, len(testFlexData[i]))

print()
print("Extension Statistics")

for i in trainExtnData.keys():
    print(i, len(trainExtnData[i]))
print(np.max(trainExtnData["current"]))
print(np.min(trainExtnData["current"]))
print()

for i in testExtnData.keys():
    print(i, len(testExtnData[i]))

In [ ]:
class GeneratorDataset(Dataset):
    def __init__(self, dataMap, device="cpu"):
        self.x = torch.tensor([dataMap["height"], dataMap["weight"]])
        # , dataMap["action"] # add to above tensor
        self.y = torch.tensor(dataMap["current"])
        self.x = self.x.T
        assert self.x.shape[0] == self.y.shape[0], f"X shape: {len(self.x)}, Y shape: {len(self.y)}"
        self.x = self.x.to(device).to(torch.float)
        self.y = self.y.to(device).to(torch.float)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return (self.x[idx], self.y[idx])

In [ ]:
batchSize=8

genTrainFlexData = GeneratorDataset(trainFlexData, device)
genTrainFlexDataloader = DataLoader(genTrainFlexData, shuffle=True, batch_size=batchSize)

genTestFlexData = GeneratorDataset(testFlexData, device)
genTestFlexDataloader = DataLoader(genTestFlexData, shuffle=True, batch_size=batchSize)


genTrainExtnData = GeneratorDataset(trainExtnData, device)
genTrainExtnDataloader = DataLoader(genTrainExtnData, shuffle=True, batch_size=batchSize)

genTestExtnData = GeneratorDataset(testExtnData, device)
genTestExtnDataloader = DataLoader(genTestExtnData, shuffle=True, batch_size=batchSize)


print(genTrainFlexData.x.shape, genTrainFlexData.y.shape, genTestFlexData.x.shape, genTestFlexData.y.shape)

trainDataset=AngleDataset(trainData, device)
trainDL = DataLoader(trainDataset, shuffle=True, batch_size=16)
print(len(trainDataset))

testDataset = AngleDataset(testData, device)
testDL = DataLoader(testDataset, shuffle=True, batch_size=16)
print(len(testDataset))

In [ ]:
class Generator(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize=1):
        super().__init__()

        self.l1 = nn.Linear(inputSize, hiddenSize, bias=True)
        self.l2 = nn.Linear(hiddenSize, hiddenSize, bias=True)
        self.l3 = nn.Linear(hiddenSize, outputSize, bias=True)

        self.a1 = nn.ReLU()
        self.a2 = nn.ReLU()

    def forward(self, x):
        o1 = self.a1(self.l1(x))
        o2 = self.a2(self.l2(o1))

        return self.l3(o2)

class Discriminator(nn.Module):
    def __init__(self, inputSize, hiddenSize, outputSize):
        super().__init__()

        self.l1 = nn.Linear(inputSize, hiddenSize, bias=True)
        self.l2 = nn.Linear(hiddenSize, hiddenSize, bias=True)
        self.l3 = nn.Linear(hiddenSize, outputSize, bias=False)

        self.a1 = nn.ReLU()
        self.a2 = nn.ReLU()
        self.a3 = nn.Sigmoid()

    def forward(self, x):
        o1 = self.a1(self.l1(x))
        o2 = self.a2(self.l2(o1))

        return self.a3(self.l3(o2))

class CoE:
    def __init__(self):
        self.flexionGenerator = None
        self.extensionGenerator = None

    def load(self, flxGen, extGen):
        self.flexionGenerator = flxGen
        self.extensionGenerator = extGen
        self.flexionGenerator.eval()
        self.extensionGenerator.eval()

    def __call__(self, x, action):
        """
        action == 1 => flexion
        action == 0 => extension
        """
        if action == 1:
            return self.flexionGenerator(x)
        else:
            return self.extensionGenerator(x)
        

In [ ]:
epochs = 200
lr=0.001
noiseSize=34
genLossFn = torch.nn.BCELoss()
recLossFn = torch.nn.MSELoss()
disLossFn = torch.nn.BCELoss()

In [ ]:
def train(genNet, disNet, genTrainLoader, genTestLoader, genOPT, disOPT, genLossFn, disLossFn, recLossFn, epochs):
    """
    returns train loss, test loss, best epoch and the best generator network
    """
    disLoss = []
    genLoss = []
    supLoss = []
    testGenError = []
    disAcc=[]
    minTestLoss = 10000
    minTrainLoss = 10000
    bestGenerator = None
    for e in tqdm(range(epochs)):
        genNet.train()
        disNet.train()
        rDisLoss = 0
        rGenLoss = 0
        rSupLoss = 0
        rDisAcc=0
        totalAllSamples=0
        for gIN, gOUT in genTrainLoader:
            # training discriminator
            disNet.zero_grad()
            genNet.zero_grad()
            
            realLabels = torch.ones((gIN.shape[0], 1)).to(device)
            
            gINWithNoise = torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
            gOUTHat = genNet(gINWithNoise)
            
            genrLabels = torch.zeros(gIN.shape[0], 1).to(device)
            
            allSamples = torch.cat((gOUT, gOUTHat))
            allSamplesLabels = torch.cat((realLabels, genrLabels))
            
            disPred = disNet(allSamples)

            rDisAcc+=(disPred.round()==allSamplesLabels).sum().item()
            totalAllSamples+=torch.numel(disPred)
            predLoss = disLossFn(disPred, allSamplesLabels)
            rDisLoss += predLoss.detach().cpu().item()
            predLoss.backward()
            disOPT.step()
    
            # # training generator
            genNet.zero_grad()
            gOUTHat1 = genNet(gINWithNoise)
    
            disOUTHat1 = disNet(gOUTHat1)
            
            unSupLoss = genLossFn(disOUTHat1, realLabels)
            rGenLoss += unSupLoss.detach().cpu().item()
            unSupLoss.backward()
            genOPT.step()

            # training generator with supervised loss            
            genNet.zero_grad()
            gOUTHat2 = genNet(gINWithNoise)
            mseLoss = recLossFn(gOUTHat2, gOUT)
            rSupLoss += mseLoss.detach().cpu().item()
            mseLoss.backward()
            genOPT.step()
        disLoss.append(rDisLoss)
        genLoss.append(rGenLoss)
        supLoss.append(rSupLoss)
        disAcc.append(rDisAcc/totalAllSamples)
        genNet.eval()
        testLoss = 0
        for gIN, gOUT in genTestLoader:
            gIN = torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
            gOUTHat = genNet(gIN)
            testLoss += torch.abs(gOUTHat - gOUT).sum()
        testGenError.append((testLoss / len(genTestFlexData)).item())
        if minTestLoss > testGenError[-1]:
            bestGenerator = deepcopy(genNet)
            minTestLoss = testGenError[-1]
            epoch = e
    return genLoss, disLoss, supLoss,disAcc, testGenError, e, bestGenerator

## Flexion Generator

In [ ]:
genFlex = Generator(36, 256, 34).to(device)
disFlex = Discriminator(34, 256, 1).to(device)
# recFlex = Reconstructor(34, 512, 2).to(device)


genOPTFlex = O.Adam(genFlex.parameters(), lr=lr)
disOPTFlex = O.Adam(disFlex.parameters(), lr=lr)
# recOPTFlex = O.Adam(recFlex.parameters(), lr=lr)
# sch = O.lr_scheduler.StepLR(, step_size=10, gamma=0.5)

# gen = Generator(36, 256, 34).to(device)
# dis = Discriminator(34, 256, 1).to(device)
# genOPT = O.Adam(gen.parameters(), lr=lr)
# disOPT = O.Adam(dis.parameters(), lr=lr)

In [ ]:
trainGenLoss, trainDisLoss, trainSupLoss,disAcc, testGenLoss, bestFlexEpoch, bestFlexGenerator = train(genFlex, disFlex, genTrainFlexDataloader, genTestFlexDataloader, genOPTFlex, disOPTFlex, genLossFn, disLossFn, recLossFn, epochs)


In [ ]:
plt.plot(trainGenLoss[0:], label="Train Loss")
plt.plot(testGenLoss[0:], label="Test Error")
plt.title("Flexion Data")
plt.legend()
plt.show()

plt.plot(disAcc)

print(f"Best epoch at {bestFlexEpoch}")
print(f"Least Train Error: {min(trainGenLoss):.3f}")
print(f"Least Test Error: {min(testGenLoss):.3f}")
print()

# print(f"Train Error %: {minTrainError*100/24.45:.3f}")
# print(f"Test  Error %: {minTestError*100/24.45:.3f}")


In [ ]:
testLoss = 0
for gIN, gOUT in genTestFlexDataloader:
    gIN=torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
    gOUTHat = bestFlexGenerator(gIN)
    testLoss += torch.abs(gOUTHat - gOUT).sum()

pointLoss = (testLoss /(len(genTestFlexDataloader.dataset)* len(genTestFlexData[0][1]))).item()
print("Mean devaition:", pointLoss)
# print("Mean Error %", pointLoss )


height = 159.0
weight = 68.0

genFlex.eval()
testInput = torch.cat((torch.tensor([height, weight]), torch.randn((noiseSize)))).to(device)
act1Curve = bestFlexGenerator(testInput ).to(device)


plt.plot(act1Curve.detach().cpu().tolist(), label="genr")
plt.plot(gOUT[2].cpu(), label="og data")
plt.legend()
plt.show()

## Extension Generator

In [ ]:
genExtn = Generator(36, 256, 34).to(device)
disExtn = Discriminator(34, 256, 1).to(device)

genOPTExtn = O.Adam(genExtn.parameters(), lr=lr)
disOPTExtn = O.Adam(disExtn.parameters(), lr=lr)
# sch = O.lr_scheduler.StepLR(, step_size=10, gamma=0.5)

In [ ]:
trainGenExtnLoss, trainDisExtnLoss, trainSupExtnLoss, disExtnLoss, testGenExtnLoss, bestExtnEpoch, bestExtnGenerator = train(genExtn, disExtn, genTrainExtnDataloader, genTestExtnDataloader, genOPTExtn, disOPTExtn, genLossFn, disLossFn, recLossFn, epochs)


In [ ]:
plt.plot(trainGenExtnLoss[0:], label="Train Loss")
plt.plot(testGenExtnLoss[0:], label="Test Error")
plt.title("Extnion Data")
plt.legend()
plt.show()

plt.plot(disAcc)

print(f"Best epoch at {bestExtnEpoch}")
print(f"Least Train Error: {min(trainGenExtnLoss):.3f}")
print(f"Least Test Error: {min(testGenExtnLoss):.3f}")
print()

In [ ]:
testLoss = 0
for gIN, gOUT in genTestExtnDataloader:
    gIN=torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
    gOUTHat = bestExtnGenerator(gIN)
    testLoss += torch.abs(gOUTHat - gOUT).sum()

pointLoss = (testLoss / len(genTestExtnData[0][1])).item()
print("Mean devaition:", pointLoss)
# print("Mean Error %", pointLoss/ )


height = 159.0
weight = 68.0
action1 = 1.0
action0 = 0.0


genExtn.eval()
testInput = torch.cat((torch.tensor([height, weight]), torch.randn((noiseSize)))).to(device)
act1Curve = bestExtnGenerator(testInput ).to(device)


plt.plot(act1Curve.detach().cpu().tolist(), label="genr")
plt.plot(gOUT[0].cpu(), label="og data")
plt.legend()

## K Fold Validation

In [ ]:
results = []
epochs = 200
lr=0.001
noiseSize=34
batchSize = 8
testBatchSize=1
genLossFn = torch.nn.BCELoss()
recLossFn = torch.nn.MSELoss()
disLossFn = torch.nn.BCELoss()
for person in ["Person A","Person B","Person G","Person D","Person E","Person F","Person H","Person I","Person J", "Person C"]:
    print("*"*50)
    print(f"Test Subject: {person}")
    personList = list(whMapping.keys())
    personList.remove(person)


    trainFlexData = {"weight":[], "height":[], "current":[]
                 ,"action":[]
                }
    
    testFlexData = {"weight":[], "height":[], "current":[]
                 ,"action":[]
                }
    
    trainExtnData = {"weight":[], "height":[], "current":[]
                 ,"action":[]
                }
    
    testExtnData = {"weight":[], "height":[], "current":[]
                 ,"action":[]
                }
    
    def get_cmap(n, name='hsv'):
        '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
        RGB color; the keyword argument name must be a standard mpl colormap name.'''
        return plt.get_cmap(name, n)
    
    
    cmap = get_cmap(9)
    for idx, i in enumerate(personList):
        for sec in sections:
            if sec[2]==0:
                continue
            sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
            # plt.plot(sig, color=cmap(idx))
            trainFlexData["current"].append(sig)
            trainFlexData["height"].append(whMapping[i][1])
            trainFlexData["weight"].append(whMapping[i][0])
            trainFlexData["action"].append(sec[2])
    
    
    for i in list([person]):
        for sec in sections:
            if sec[2] == 0:
                continue
            sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
            testFlexData["current"].append(sig)
            testFlexData["height"].append(whMapping[i][1])
            testFlexData["weight"].append(whMapping[i][0])
            testFlexData["action"].append(sec[2])
    
    for idx, i in enumerate(personList):
        for sec in sections:
            if sec[2]==1:
                continue
            sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
            # plt.plot(sig, color=cmap(idx))
            trainExtnData["current"].append(sig)
            trainExtnData["height"].append(whMapping[i][1])
            trainExtnData["weight"].append(whMapping[i][0])
            trainExtnData["action"].append(sec[2])
    
    
    for i in list([person]):
        for sec in sections:
            if sec[2] == 1:
                continue
            sig = butter_lowpass_filter(rawData[i][sec[0]:sec[1]], cutoff, fs, order)[5:]
            testExtnData["current"].append(sig)
            testExtnData["height"].append(whMapping[i][1])
            testExtnData["weight"].append(whMapping[i][0])
            testExtnData["action"].append(sec[2])
    
    
    
    trainFlexData["current"] = np.array(trainFlexData["current"]) / 1000
    testFlexData["current"] = np.array(testFlexData["current"]) / 1000
    
    trainExtnData["current"] = np.array(trainExtnData["current"]) / 1000
    testExtnData["current"] = np.array(testExtnData["current"]) / 1000
    
    
    # assert len(trainData["current"]) == len(trainData["height"]) == len(trainData["weight"]) == len(trainData["action"]) == len(trainData["angle"]), "train data not homogeneous"
    # assert len(testData["current"]) == len(testData["height"]) == len(testData["weight"]) == len(testData["action"]) == len(testData["angle"]), "test data not homogeneous"

    print(f"Train Len: {len(trainFlexData["current"])}, Test Len: {len(testFlexData["current"])}")

    genTrainFlexData = GeneratorDataset(trainFlexData, device)
    genTrainFlexDataloader = DataLoader(genTrainFlexData, shuffle=True, batch_size=batchSize)
    
    genTestFlexData = GeneratorDataset(testFlexData, device)
    genTestFlexDataloader = DataLoader(genTestFlexData, shuffle=True, batch_size=testBatchSize)
    
    
    genTrainExtnData = GeneratorDataset(trainExtnData, device)
    genTrainExtnDataloader = DataLoader(genTrainExtnData, shuffle=True, batch_size=batchSize)
    
    genTestExtnData = GeneratorDataset(testExtnData, device)
    genTestExtnDataloader = DataLoader(genTestExtnData, shuffle=True, batch_size=testBatchSize)

    print(genTrainFlexData.x.shape)
########### Flex Train and Test


    genFlex = Generator(36, 256, 34).to(device)
    disFlex = Discriminator(34, 256, 1).to(device)    
    
    genOPTFlex = O.Adam(genFlex.parameters(), lr=lr)
    disOPTFlex = O.Adam(disFlex.parameters(), lr=lr)

    trainGenFlexLoss, trainDisFlexLoss, trainSupFlexLoss, disAccFlex, testGenFlexLoss, bestFlexEpoch, bestFlexGenerator = train(genFlex, disFlex, genTrainFlexDataloader, genTestFlexDataloader, genOPTFlex, disOPTFlex, genLossFn, disLossFn, recLossFn, epochs)
    trainGenExtnLoss, trainDisExtnLoss, trainSupExtnLoss, disAccExtn, testGenExtnLoss, bestExtnEpoch, bestExtnGenerator = train(genExtn, disExtn, genTrainExtnDataloader, genTestExtnDataloader, genOPTExtn, disOPTExtn, genLossFn, disLossFn, recLossFn, epochs)

    plt.plot(trainGenFlexLoss[0:], label="Flex Train Error")
    plt.plot(testGenFlexLoss[0:], label="Flex Test Error")
    plt.plot(trainGenExtnLoss, label="Extn Train Error")
    plt.plot(testGenExtnLoss, label="Extn Test Error")
    plt.title(f"Train and Test Loss for {person}")
    plt.legend()
    plt.show()

    plt.plot(disAccFlex, label="Flex Disc Acc")
    plt.plot(disAccExtn, label="Extn Disc Acc")
    plt.legend()
    plt.show()
    print(f"Best Flex Train epoch at {bestFlexEpoch}")
    print(f"Least Flex Train Error: {min(trainGenFlexLoss):.3f}")
    print(f"Least Flex Test Error: {min(testGenFlexLoss):.3f}")
    print(f"Avg Flex Dis Acc in last 20 epochs: {sum(disAccFlex[-20:])/20:.3f}")
    print()

    print(f"Best Extn Train epoch at {bestExtnEpoch}")
    print(f"Least Extn Train Error: {min(trainGenExtnLoss):.3f}")
    print(f"Least Extn Test Error: {min(testGenExtnLoss):.3f}")
    print(f"Avg Extn Dis Acc in last 20 epochs: {sum(disAccExtn[-20:])/20:.3f}")
    print()
    

    testFlexLoss = 0
    testExtnLoss = 0
    for gIN, gOUTFlex in genTestFlexDataloader:
        gIN=torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
        gOUTFlexHat = bestFlexGenerator(gIN)
        testFlexLoss += torch.abs(gOUTFlexHat - gOUTFlex).sum()
    for gIN, gOUTExtn in genTestExtnDataloader:
        gIN = torch.cat((gIN, torch.randn((gIN.shape[0], noiseSize)).to(device)), dim=1)
        gOUTExtnHat = bestExtnGenerator(gIN)
        testExtnLoss+= torch.abs(gOUTExtnHat - gOUTExtn).sum()

    pointFlexLoss = (testFlexLoss /(len(genTestFlexDataloader.dataset)* len(genTestFlexData[0][1]))).item()
    pointExtnLoss = (testExtnLoss /(len(genTestExtnDataloader.dataset)* len(genTestExtnData[0][1]))).item()
    print("Mean flex devaition for the test patient:", pointFlexLoss)
    print("Mean extn devaition for the test patient:", pointExtnLoss)
    
    genFlex.eval()
    genExtn.eval()
    
    testInput = gIN
    act1Curve = bestFlexGenerator(testInput).to(device)
    act0Curve = bestExtnGenerator(testInput).to(device)
    plt.plot(act0Curve.flatten().detach().cpu().tolist(), label="Extn Pred")
    plt.plot(act1Curve.flatten().detach().cpu().tolist(), label="Flex Pred")
    plt.plot(gOUTFlex.flatten().detach().cpu().tolist(), label="Flex Orig")
    plt.plot(gOUTExtn.flatten().detach().cpu().tolist(), label="Extn Orig")
    # plt.plot(gOUT[2].cpu(), label="OG Data")
    plt.title(f"Test: {person}")
    plt.legend()
    plt.xlabel("Time Steps")
    plt.ylabel("Power in W")
    plt.grid()
    plt.show()
    ################
    results.append([person, bestFlexEpoch, pointFlexLoss, sum(disAccFlex[-20:])/20, bestExtnEpoch, pointExtnLoss, sum(disAccExtn[-20:])/20])

In [ ]:
rsPD  = pd.DataFrame(results)
rsPD.columns = columns=["Person", "BestFlexEpoch", "flexAvgLoss", "FlexAvgAcc", "BestExtnEpoch", "extnAvgLoss", "ExtnAvgAcc"]
print(rsPD)

In [ ]:
for i in range(0, 10):
    print(rsPD.loc[i]["Person"], ":", "{:.3f}".format((rsPD.loc[i]["flexAvgLoss"]+ rsPD.loc[i]["extnAvgLoss"])/2))